In [1]:
# Mount your Google Drive.
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# ! pip install kaggle

In [3]:
# ! mkdir ~/.kaggle
# ! cp /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !cd '/content/drive/MyDrive/cancer cell classification'
# ! kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
# ! mkdir '/content/drive/MyDrive/cancer cell classification/dataset'
# ! unzip /content/skin-cancer-mnist-ham10000.zip -d '/content/drive/MyDrive/cancer cell classification/dataset'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import itertools
import cv2

from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model


In [5]:
df = pd.read_csv('/content/drive/MyDrive/cancer cell classification/dataset/HAM10000_metadata.csv')

In [6]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [7]:
df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age             57
sex              0
localization     0
dtype: int64

In [8]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
base_skin_dir = '/content/drive/MyDrive/cancer cell classification/dataset'

In [9]:
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

In [10]:
df['path'] = df['image_id'].map(imageid_path_dict.get)
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes

In [11]:
df.head(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/content/drive/MyDrive/cancer cell classificat...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/content/drive/MyDrive/cancer cell classificat...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/content/drive/MyDrive/cancer cell classificat...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/content/drive/MyDrive/cancer cell classificat...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/content/drive/MyDrive/cancer cell classificat...,Benign keratosis-like lesions,2


In [12]:
df['age'].fillna((df['age'].mean()), inplace=True)

In [13]:
df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age              0
sex              0
localization     0
path             0
cell_type        0
cell_type_idx    0
dtype: int64

In [16]:
df['image'] = ''

In [ ]:
for i in range(0,len(df)):
  print(i)
  df['image'][i] = cv2.resize(cv2.imread(df['path'][i]),(75,75))


In [19]:
df.tail()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,image
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,/content/drive/MyDrive/cancer cell classificat...,Actinic keratoses,0,"[[[152, 142, 157], [144, 133, 150], [128, 116,..."
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,/content/drive/MyDrive/cancer cell classificat...,Actinic keratoses,0,"[[[3, 6, 6], [5, 7, 7], [5, 7, 8], [12, 11, 13..."
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,/content/drive/MyDrive/cancer cell classificat...,Actinic keratoses,0,"[[[108, 105, 117], [123, 121, 136], [130, 132,..."
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,/content/drive/MyDrive/cancer cell classificat...,Actinic keratoses,0,"[[[146, 124, 158], [130, 115, 155], [144, 127,..."
10014,HAM_0003521,ISIC_0032258,mel,histo,70.0,female,back,/content/drive/MyDrive/cancer cell classificat...,Melanoma,5,"[[[141, 152, 181], [127, 141, 177], [128, 147,..."


In [22]:
n_samples = 5
fig, m_axs = plt.subplots(7, n_samples, figsize = (4*n_samples, 3*7))
for n_axs, (type_name, type_rows) in zip(m_axs, df.sort_values(['cell_type']).groupby('cell_type')):
    n_axs[0].set_title(type_name)
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=1234).iterrows()):
        c_ax.imshow(c_row['image'])
        c_ax.axis('off')
fig.savefig('/content/drive/MyDrive/cancer cell classification/category_samples.png', dpi=300)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
features=df['image']
target=df['cell_type_idx']

In [24]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20,random_state=1234)

In [25]:
x_train = np.asarray(x_train.tolist())
x_test = np.asarray(x_test.tolist())

In [26]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

In [27]:
# x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)

In [28]:
x_train = x_train.reshape(x_train.shape[0], *(75, 75, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 75, 3))
# x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [29]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False ## Not trainable weights

## Preprocessing input
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)
# x_validate = preprocess_input(x_validate)

58889256/58889256 [==============================] - 0s 0us/step


In [30]:
base_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 75, 75, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 75, 75, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 75, 75, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 37, 37, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 37, 37, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 37, 37, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 18, 128)       0     

In [31]:
from tensorflow.keras import layers, models

In [32]:
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(128, activation='relu')
prediction_layer = layers.Dense(7, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer,
    prediction_layer
])

In [33]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 14,977,863
Trainable params: 263,175
Non-trainable params: 14,714,688
_________________________________________________________________


In [34]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [35]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


# es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)



In [57]:
model.fit(x_train, y_train, epochs=25, validation_split=0.2, batch_size=32)

Epoch 1/25
201/201 [==============================] - 6s 31ms/step - loss: 0.2872 - accuracy: 0.8987 - val_loss: 1.2063 - val_accuracy: 0.7149
Epoch 2/25
201/201 [==============================] - 6s 29ms/step - loss: 0.2641 - accuracy: 0.9056 - val_loss: 1.3038 - val_accuracy: 0.7305
Epoch 3/25
201/201 [==============================] - 6s 29ms/step - loss: 0.1789 - accuracy: 0.9377 - val_loss: 1.3028 - val_accuracy: 0.7080
Epoch 4/25
201/201 [==============================] - 6s 30ms/step - loss: 0.1089 - accuracy: 0.9665 - val_loss: 1.3735 - val_accuracy: 0.7012
Epoch 5/25
201/201 [==============================] - 6s 29ms/step - loss: 0.0742 - accuracy: 0.9789 - val_loss: 1.4128 - val_accuracy: 0.7218
Epoch 6/25
201/201 [==============================] - 6s 30ms/step - loss: 0.0450 - accuracy: 0.9891 - val_loss: 1.6066 - val_accuracy: 0.7305
Epoch 7/25
201/201 [==============================] - 6s 29ms/step - loss: 0.0330 - accuracy: 0.9919 - val_loss: 1.6576 - val_accuracy: 0.7268

In [58]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
# loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
# print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("/content/drive/MyDrive/cancer cell classification/model.h5")

63/63 [==============================] - 2s 24ms/step - loss: 2.5017 - accuracy: 0.7324
Test: accuracy = 0.732401  ;  loss = 2.501726


In [59]:
import tensorflow as tf

In [60]:
class_names = ['Melanocytic nevi','Melanoma','Benign keratosis-like lesions ','Basal cell carcinoma','Actinic keratoses','Vascular lesions','Dermatofibroma']

In [62]:
img_ = "/content/drive/MyDrive/cancer cell classification/dataset/ham10000_images_part_1/ISIC_0027419.jpg"

img = tf.keras.utils.load_img(
    img_, target_size=(75, 75)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 19ms/step
This image most likely belongs to Benign keratosis-like lesions  with a 23.99 percent confidence.
